In [1]:
import pandas as pd


from google.colab import drive
drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/2018.1manhattenyellowtaxis.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [3]:
print(len(df))

99177964


In [4]:
df['tpep_pickup_datetime']  = pd.to_datetime(df['tpep_pickup_datetime'] ,  format ='%m/%d/%Y %I:%M:%S %p')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'] ,  format ='%m/%d/%Y %I:%M:%S %p')

In [5]:
print(df.dtypes)

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object


In [6]:
print(df.isnull().sum())

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64


dealing with negative values

In [7]:
x = [
    "payment_type", "VendorID", "passenger_count", "trip_distance",
    "RatecodeID","PULocationID","DOLocationID","fare_amount",
    "extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge",
    "total_amount"
]


d = {}


for feature in x:
    if feature in df.columns:
        d[feature] = (df[feature] < 0).sum()

for feature,  count in d.items():
    print(f'{feature}: {count}')

payment_type: 0
VendorID: 0
passenger_count: 0
trip_distance: 0
RatecodeID: 0
PULocationID: 0
DOLocationID: 0
fare_amount: 58779
extra: 29015
mta_tax: 57686
tip_amount: 514
tolls_amount: 391
improvement_surcharge: 58748
total_amount: 58781


In [8]:
for i in x:
    if i in df.columns:
        df[i] = df[i].apply(lambda y: abs(y) if y < 0 else y)

In [9]:
x = [
    "payment_type", "VendorID", "passenger_count", "trip_distance",
    "RatecodeID","PULocationID","DOLocationID","fare_amount",
    "extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge",
    "total_amount"
]


d = {}


for feature in x:
    if feature in df.columns:
        d[feature] = (df[feature] < 0).sum()

for feature,  count in d.items():
    print(f'{feature}: {count}')

payment_type: 0
VendorID: 0
passenger_count: 0
trip_distance: 0
RatecodeID: 0
PULocationID: 0
DOLocationID: 0
fare_amount: 0
extra: 0
mta_tax: 0
tip_amount: 0
tolls_amount: 0
improvement_surcharge: 0
total_amount: 0


Next we are going to remove values that are incoherent based on domain knowledge. As discussed in the data quality report NYC taxis can only take 6 passengers including a child under 7 and the base fare for 2018 was 2.50$

we will count hte number of rows with incoherent values. If many compared to the large  dataset we may choose imputation or if there are only a few rows we may just remove them.

References

Bloomberg (2022) 'NYC Taxi Cab Fares to Rise 23% in First Increase Since 2012', Bloomberg. Available at: https://www.bloomberg.com/news/articles/2022-11-15/nyc-taxi-cab-fares-to-rise-23-in-firs t-increase-since-2012 (Accessed: 7 July 2023).
NYC.gov (n.d.) 'TLC Rules', NYC.gov. Available at: https://www.nyc.gov/site/tlc/about/tlc-rules.page (Accessed: 7 July 2023).

In [10]:
o1 = df[df['passenger_count'] > 6 ].shape[0]
print(o1)

500


In [11]:
df = df[ df['passenger_count']  <= 6]

In [12]:
o1 = df[df['passenger_count'] > 6 ].shape[0]
print(o1)

0


In [13]:
o2 = df[df['passenger_count'] == 0].shape[0]
print(o2)

848284


In [14]:
df['passenger_count'].replace(0 , 1 ,  inplace=True)

In [15]:
o2 = df[df['passenger_count'] == 0].shape[0]
print(o2)

0


In [16]:
o3 = (df['tpep_pickup_datetime']  >=  df['tpep_dropoff_datetime']).sum()
print(o3)

5198


In [17]:
df =   df[df['tpep_pickup_datetime']   < df['tpep_dropoff_datetime']]

In [18]:
start = pd.Timestamp('2018-01-01')
end = pd.Timestamp('2018-12-31')

In [19]:
out = ((df['tpep_pickup_datetime'] < start)  | (df['tpep_pickup_datetime'] > end)) | ((df['tpep_dropoff_datetime'] < start) |  (df['tpep_dropoff_datetime']  > end))
print(out.sum())

186473


In [20]:
df = df[((df['tpep_pickup_datetime']  >= start)    & (df['tpep_pickup_datetime']  <= end))   & ((df['tpep_dropoff_datetime'] >= start)    & (df['tpep_dropoff_datetime']  <= end))]

In [21]:
out = ((df['tpep_pickup_datetime'] < start)  | (df['tpep_pickup_datetime'] > end)) | ((df['tpep_dropoff_datetime'] < start) |  (df['tpep_dropoff_datetime']  > end))
print(out.sum())

0


In [22]:
print((df['VendorID'] == 4).sum())

440216


In [23]:
print((df['RatecodeID'] == 99).sum())

600


In [24]:
df['RatecodeID'].replace(99 , 1 , inplace=True)

In [25]:
print((df['RatecodeID'] == 99).sum())

0


In [26]:
print(len(df))

98985793


In [27]:
print(df['improvement_surcharge'].unique())

[3.0000e-01 0.0000e+00 4.0003e+03]


0.03 is likely a mistake since data dictionary states its a standard 30% tax.

In [28]:
print((df['improvement_surcharge'] == 4.0003e+03).sum())

2


In [29]:
df['improvement_surcharge'].replace(4.0003e+03 , 0.3 , inplace=True)

In [30]:
print((df['improvement_surcharge'] == 4.0003e+03 ).sum())

0


In [31]:
print(df['mta_tax'].unique())

[ 0.    0.5   0.35  2.31  0.32  0.6  15.49  1.17  0.87 80.8   0.89 21.5
  6.42 33.63  5.9  18.32 60.8 ]


In [32]:
mode_mta_tax = df['mta_tax'].mode()[0]

# mta tax is a standard 50 cent charge
df.loc[~df['mta_tax'].isin([0, 0.50]), 'mta_tax'] = mode_mta_tax

In [33]:

def detectouts(z):
    Q1 = z.quantile(0.25)
    Q3 = z.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((z < (Q1 - 1.5 * IQR)) | (z > (Q3 + 1.5 * IQR)))
    return outliers.sum()

outcols1 = ['trip_distance', 'fare_amount', 'tip_amount',  'tolls_amount', 'total_amount']


outcounts = {col: detectouts(df[col]) for col in outcols1}

outcounts


{'trip_distance': 8424515,
 'fare_amount': 6724112,
 'tip_amount': 3603089,
 'tolls_amount': 3407796,
 'total_amount': 6598549}

In [34]:
def cappy(df, w):


    Q1 = df[w].quantile(0.25)
    Q3 = df[w].quantile(0.75)

    #subtract the 2
    IQR = Q3 - Q1


    lower_b = Q1 - 1.5 * IQR
    upper_b = Q3 + 1.5 * IQR


    df[w] = df[w].clip(lower=lower_b, upper=upper_b)

    return df

outcols = ['trip_distance',   'fare_amount', 'tip_amount',   'tolls_amount', 'total_amount']

for col in outcols:


    df =  cappy(df, col)

df.head()



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-04-27 20:15:08,2018-04-27 20:15:33,1,0.0,5,N,143,143,1,22.75,0.0,0.0,5.65,0.0,0.3,27.425
1,1,2018-04-27 20:22:24,2018-04-27 20:34:10,1,1.9,1,N,48,68,2,10.00,0.5,0.5,0.00,0.0,0.3,11.300
2,1,2018-04-27 20:38:44,2018-04-27 20:52:05,1,1.5,1,N,90,170,1,10.00,0.5,0.5,2.80,0.0,0.3,14.100
3,1,2018-04-27 20:54:27,2018-04-27 20:59:53,1,0.9,1,N,170,79,2,6.00,0.5,0.5,0.00,0.0,0.3,7.300
4,1,2018-04-27 20:08:20,2018-04-27 20:12:44,2,0.6,1,N,237,237,2,5.00,0.5,0.5,0.00,0.0,0.3,6.300


In [35]:
print(df.duplicated().sum())

8337182


In [36]:
print(len(df))

98985793


In [37]:
df.drop_duplicates(inplace=True)

In [38]:
print(len(df))

90648611


In [39]:
df.to_csv('/content/drive/MyDrive/2018.22manhattenyellowtaxis.csv', index=False)